In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
df = pd.DataFrame()
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv', encoding='utf-8')

df.head(3)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive


In [4]:
df.loc[df['sentiment'] == 'positive', 'sentiment'] = 1
df.loc[df['sentiment'] == 'negative', 'sentiment'] = 0


In [5]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [6]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

review_lines = list()
lines = df['review'].values.tolist()

for line in lines:   
    tokens = word_tokenize(line)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word    
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words    
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    review_lines.append(words)

In [7]:
len(review_lines)

50000

In [8]:
import gensim 

EMBEDDING_DIM = 100
# train word2vec model
model = gensim.models.Word2Vec(sentences=review_lines, size=EMBEDDING_DIM, window=5, workers=4, min_count=1)
# vocab size
words = list(model.wv.vocab)
print('Vocabulary size: %d' % len(words))

Vocabulary size: 134156


In [9]:
# save model in ASCII (word2vec) format
filename = 'imdb_embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)

In [10]:
# let us try some utility functions of gensim word2vec more details here 

model.wv.most_similar('horrible')#, topn =1)

[('terrible', 0.9369164705276489),
 ('awful', 0.8719069957733154),
 ('horrid', 0.8024217486381531),
 ('atrocious', 0.7962092161178589),
 ('horrendous', 0.7941433191299438),
 ('pathetic', 0.7894285917282104),
 ('dreadful', 0.7854096293449402),
 ('lousy', 0.7846511006355286),
 ('sucks', 0.782234787940979),
 ('laughable', 0.742453932762146)]

In [11]:
#Let’s see the result of semantically reasonable word vectors (king - man + woman)
model.wv.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])

[('juliet', 0.8808293342590332),
 ('romeo', 0.8786265850067139),
 ('princess', 0.850227952003479),
 ('tromeo', 0.8299891352653503),
 ('lion', 0.8298365473747253),
 ('bride', 0.8290273547172546),
 ('lennie', 0.8252159953117371),
 ('prince', 0.8228528499603271),
 ('queen', 0.8216102719306946),
 ('cordaraby', 0.8215664029121399)]

In [12]:

#Let’s see the result of semantically reasonable word vectors (king - man + woman)
model.wv.most_similar(positive=['woman', 'king'], negative=['man'])

[('lion', 0.6205703616142273),
 ('princess', 0.5915178656578064),
 ('juliet', 0.5899780988693237),
 ('romeo', 0.58808434009552),
 ('bride', 0.5648108124732971),
 ('rice', 0.5589447021484375),
 ('hawkings', 0.5551849603652954),
 ('lear', 0.555041491985321),
 ('queen', 0.5493825674057007),
 ('cordaraby', 0.539921760559082)]

In [13]:
#odd word out
print(model.wv.doesnt_match("woman king queen movie".split()))

movie


/opt/conda/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


In [14]:
model.wv.similar_by_word("cat")

[('dog', 0.7618103623390198),
 ('hat', 0.7261778116226196),
 ('mouse', 0.7034851908683777),
 ('soup', 0.6988158226013184),
 ('chicken', 0.6888816952705383),
 ('snake', 0.6759913563728333),
 ('rabbit', 0.6749992370605469),
 ('dude', 0.6657153367996216),
 ('monkey', 0.6647992134094238),
 ('bugs', 0.6627668142318726)]

In [15]:
print(model.similarity('boy', 'girl'))

0.8303565


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


In [16]:
import os

embeddings_index = {}
f = open(os.path.join('', 'imdb_embedding_word2vec.txt'),  encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()

In [17]:
X_train = df.loc[:24999, 'review'].values
y_train = df.loc[:24999, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [18]:
total_reviews = X_train + X_test
max_length = 100 # try other options like mean of sentence lengths

In [19]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

VALIDATION_SPLIT = 0.2

# vectorize the text samples into a 2D integer tensor
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(review_lines)
sequences = tokenizer_obj.texts_to_sequences(review_lines)

# pad sequences
word_index = tokenizer_obj.word_index
print('Found %s unique tokens.' % len(word_index))

review_pad = pad_sequences(sequences, maxlen=max_length)
sentiment =  df['sentiment'].values
print('Shape of review tensor:', review_pad.shape)
print('Shape of sentiment tensor:', sentiment.shape)

# split the data into a training set and a validation set
indices = np.arange(review_pad.shape[0])
np.random.shuffle(indices)
review_pad = review_pad[indices]
sentiment = sentiment[indices]
num_validation_samples = int(VALIDATION_SPLIT * review_pad.shape[0])

X_train_pad = review_pad[:-num_validation_samples]
y_train = sentiment[:-num_validation_samples]
X_test_pad = review_pad[-num_validation_samples:]
y_test = sentiment[-num_validation_samples:]

Found 134156 unique tokens.
Shape of review tensor: (50000, 100)
Shape of sentiment tensor: (50000,)


In [20]:
print('Shape of X_train_pad tensor:', X_train_pad.shape)
print('Shape of y_train tensor:', y_train.shape)

print('Shape of X_test_pad tensor:', X_test_pad.shape)
print('Shape of y_test tensor:', y_test.shape)

Shape of X_train_pad tensor: (40000, 100)
Shape of y_train tensor: (40000,)
Shape of X_test_pad tensor: (10000, 100)
Shape of y_test tensor: (10000,)


In [21]:
EMBEDDING_DIM =100
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [22]:
print(num_words)

134157


In [23]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.initializers import Constant

# define model
model = Sequential()
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_length,
                            trainable=False)

model.add(embedding_layer)
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the model
model.fit(X_train_pad, y_train, batch_size=128, epochs=25, validation_data=(X_test_pad, y_test), verbose=2)

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          13415700  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 96, 128)           64128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 48, 128)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6144)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6145      
Total params: 13,485,973
Trainable params: 70,273
Non-trainable params: 13,415,700
_________________________________________________________________
None
Train on 40000 samples, validate on 10000 samples
Epoch 1/25
 - 16s - loss: 0.4121 - accuracy: 0.8

In [24]:

# evaluate the model
loss, accuracy = model.evaluate(X_test_pad, y_test, batch_size=128)
print('Accuracy: %f' % (accuracy*100))

10000/10000 [==============================] - 1s 138us/step
Accuracy: 84.259999


In [25]:
#Let us test some  samples
# load the dataset but only keep the top n words, zero the rest

test_sample_1 = "This movie is fantastic! I really like it because it is so good!"
test_sample_2 = "Good movie!"
test_sample_3 = "Maybe I like this movie."
test_sample_4 = "Not to my taste, will skip and watch another movie"
test_sample_5 = "if you like action, then this movie might be good for you."
test_sample_6 = "Bad movie!"
test_sample_7 = "Not a good movie!"
test_sample_8 = "This movie really sucks! Can I get my money back please?"
test_samples = [test_sample_1, test_sample_2, test_sample_3, test_sample_4, test_sample_5, test_sample_6, test_sample_7, test_sample_8]

test_samples_tokens = tokenizer_obj.texts_to_sequences(test_samples)
test_samples_tokens_pad = pad_sequences(test_samples_tokens, maxlen=max_length)

#predict
model.predict(x=test_samples_tokens_pad)

array([[0.99973947],
       [0.6927332 ],
       [0.67135173],
       [0.43925798],
       [0.7572388 ],
       [0.00264442],
       [0.6927332 ],
       [0.02970614]], dtype=float32)

In [26]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding
from keras.initializers import Constant

# define model
model = Sequential()
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_length,
                            trainable=False)
model.add(embedding_layer)
model.add(GRU(units=32,  dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print('Summary of the built model...')
print(model.summary())

Summary of the built model...
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          13415700  
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                12768     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 13,428,501
Trainable params: 12,801
Non-trainable params: 13,415,700
_________________________________________________________________
None


In [27]:
print('Train...')

model.fit(X_train_pad, y_train, batch_size=128, epochs=25, validation_data=(X_test_pad, y_test), verbose=2)

Train...
Train on 40000 samples, validate on 10000 samples
Epoch 1/25
 - 24s - loss: 247.7697 - accuracy: 0.5070 - val_loss: 0.7121 - val_accuracy: 0.5134
Epoch 2/25
 - 24s - loss: 246.2586 - accuracy: 0.5077 - val_loss: 0.7086 - val_accuracy: 0.5224
Epoch 3/25
 - 23s - loss: 1562.5152 - accuracy: 0.5167 - val_loss: 0.7093 - val_accuracy: 0.5187
Epoch 4/25
 - 23s - loss: 105.1660 - accuracy: 0.5106 - val_loss: 0.7086 - val_accuracy: 0.5273
Epoch 5/25
 - 23s - loss: 0.7189 - accuracy: 0.5145 - val_loss: 0.7074 - val_accuracy: 0.5284
Epoch 6/25
 - 23s - loss: 1.3251 - accuracy: 0.5158 - val_loss: 0.7077 - val_accuracy: 0.5265
Epoch 7/25
 - 24s - loss: 0.7377 - accuracy: 0.5142 - val_loss: 0.7077 - val_accuracy: 0.5283
Epoch 8/25
 - 23s - loss: 0.7190 - accuracy: 0.5160 - val_loss: 0.7077 - val_accuracy: 0.5273
Epoch 9/25
 - 23s - loss: 0.7176 - accuracy: 0.5183 - val_loss: 0.7076 - val_accuracy: 0.5286
Epoch 10/25
 - 23s - loss: 0.8534 - accuracy: 0.5135 - val_loss: 0.7075 - val_accuracy

In [28]:
print('Testing...')
score, acc = model.evaluate(X_test_pad, y_test, batch_size=128)

print('Test score:', score)
print('Test accuracy:', acc)

print("Accuracy: {0:.2%}".format(acc))

Testing...
10000/10000 [==============================] - 1s 147us/step
Test score: 0.7024012398719788
Test accuracy: 0.535099983215332
Accuracy: 53.51%
